In [11]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', 70)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', 80)                      # 출력할 열의 너비

In [12]:
review_matrix = pd.read_csv('리뷰 속 단어 체크체크 완료.csv')
review_matrix[:16]

,id,place,count_review,avg_review,total_review,돈,쇼핑,시장,"싸다, 저렴하다",비싸다,먹다,"음식, 식사",상품,구매,가격,호텔,박물관,전시관,관광지,유적지,미술관,역사,유익하다,문화,궁전,전쟁,활동,놀이공원,게임,친구,커플,재미,아쿠아리움,엔터테인먼트,아이,가족,경기,힐링,공원,휴식,풍경,자연,전망대,산책,분수,여유,나들이,공연,콘서트,이벤트,영화,예술,미술,강좌,요리,공예,체험,tag_list
0,1,교보문고 광화문,74,4.540541,"['교보문고\n광화문역 5호선지하철역이랑 연결되어 있어서 찾아가기 아주좋아요 광화문교보문고 넓고 다양한 책들을 볼수있어 좋아오', '교보...",1,16,0,4,9,1,18,5,11,2,3,0,0,2,0,0,3,0,8,2,1,0,0,1,1,2,3,0,0,15,2,0,0,0,9,0,0,0,0,0,4,0,0,0,0,2,2,0,0,0,0,0,"['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']"
1,2,롯데월드타워&몰,203,4.206897,"""좋은 쇼핑몰\n남자친구와 함께 방문한 곳이 있습니다. 이 많은 상품을 파는 브랜드 뿐만 아니라 다른 브랜드 인식 (하지만 더 저렴한)'...",0,1,0,1,2,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']"
2,3,SM타운 코엑스아티움,66,4.181818,['뮤지컬 공연도 하고\n가끔 지나다가 들려서 구경할 때가 있는데 큰 기대는 안한다면 괜찮은 관광코스 되더라구요 지하철역에도 워낙 아이돌...,7,15,2,11,11,1,27,19,9,5,1,21,6,2,0,0,5,0,3,1,0,3,0,0,4,0,8,0,9,9,2,0,0,0,1,0,0,0,0,0,0,0,6,5,1,3,13,0,0,0,0,0,"['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']"
3,4,홍대앞 예술시장 프리마켓,75,4.386667,['홍대앞 예술시장 프리마켓 이라고 되어있구나\n홍대라고 하면 여기까지 지칭해서 따로 없을줄 알았는데 있네요 여기 그냥 홍대앞에가서 거리...,3,46,23,18,4,14,52,7,2,20,3,1,0,6,0,1,0,1,2,0,0,1,0,0,4,1,9,0,0,6,0,0,0,1,1,0,1,0,6,0,0,2,14,0,0,0,10,1,0,0,6,0,"['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']"
4,5,광장시장,634,4.113565,['생각보다는..\n워낙 사람이붐비고 유명한곳이라가봤는데 육회덮밥만먹고왔어요 생각보다 번잡스럽고 한복이나 의류상가들이 너무많아서 놀랐어요...,8,5,87,14,3,73,105,1,3,11,0,1,0,10,0,0,0,0,1,0,0,0,0,0,4,0,8,0,0,4,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,"['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']"
5,6,고투 몰,33,4.272727,"['고투몰\n지하철 3호선, 7호선, 9호선을 환승할 수 있는 고속터미널역에서 바로 갈 수 있어 접근성이 좋습니다. 다양한 종류의 물건을...",4,59,9,24,4,2,6,10,8,26,0,0,0,2,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"['쇼핑', '가격', '싸다, 저렴하다', '상품', '시장', '구매', '음식, 식사']"
6,7,신세계 백화점 본점,92,4.097826,['개인적으로 대한민국에서 가장 멋진 백화점\n신세계백화점 본점은 제 개인적으로는 대한민국에서 가장 멋진 백화점으로 생각되는 곳이예요. ...,4,164,16,15,55,17,105,17,15,13,9,0,2,6,0,0,0,0,1,0,0,0,0,0,4,0,5,0,1,7,3,0,0,1,2,0,0,0,1,1,2,0,0,0,0,0,0,0,1,5,1,1,"['쇼핑', '음식, 식사', '비싸다', '먹다', '상품', '시장', '싸다, 저렴하다']"
7,8,타임스퀘어,103,4.174757,"['편리한 쇼핑몰\n교통도 좋고 백화점과 같이 있어 여러모로 편리합니다. 카페, 레스토랑 라인업도 예전보다 좋고 풍부해진 것 같네요. 여...",1,235,16,13,23,14,101,6,2,6,11,0,0,3,0,0,0,0,2,0,0,0,0,1,2,1,5,0,4,5,2,0,0,0,5,0,0,0,1,0,1,1,2,0,4,28,0,0,0,1,0,0,"['쇼핑', '음식, 식사', '영화', '비싸다', '시장', '먹다', '싸다, 저렴하다']"
8,9,가로수길,277,3.870036,['..\n한 7년전만해도 암청 핫했는데.. 상권이 많이 죽었더라구요 임대문의 써있는 곳도 많고... 이젠 그닥 매력적이지 않은 동네.....,0,142,0,21,35,21,170,3,1,11,5,0,1,12,0,0,0,0,5,2,0,0,0,0,8,2,7,0,0,6,0,0,1,1,7,0,0,0,11,0,3,0,0,0,0,1,1,0,0,0,0,0,"['음식, 식사', '쇼핑', '비싸다', '싸다, 저렴하다', '먹다', '관광지', '산책']"
9,10,라인프렌즈 플래그십 스토어 명동역점,38,4.368421,['인생샷 건지기 좋은 분위기\n캐릭터 덕후인 내게 너무나도 좋았던 곳! 저 왕토끼가 너무 귀여워서 같이 셀카찍고 싶었는데 사람이 많아 ...,0,21,0,6,6,0,4,20,8,5,0,0,0,2,0,0,0,0,0,0,0,0,0,0,10,0,4,0,0,12,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"['쇼핑', '상품', '아이', '친구', '구매', '싸다, 저렴하다', '비싸다']"


In [35]:
review_tag_count = pd.read_csv('cbf_matrix.csv')
review_tag_count['count'] = review_matrix['count_review']
review_tag_count['avg_score'] = review_matrix['avg_review']
review_tag_count = review_tag_count.drop(['listed_tag','tag_list_literal'],axis=1)
review_tag_count

,id,place,tag_list,count,avg_score
0,1,교보문고 광화문,"['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']",74,4.540541
1,2,롯데월드타워&몰,"['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']",203,4.206897
2,3,SM타운 코엑스아티움,"['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']",66,4.181818
3,4,홍대앞 예술시장 프리마켓,"['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']",75,4.386667
4,5,광장시장,"['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']",634,4.113565
...,...,...,...,...,...
1123,2297,디보 빌리지,['쇼핑'],1,1.000000
1124,2300,서울빛초롱축제,"['음식, 식사', '관광지', '재미', '전시관', '예술', '이벤트', '산책']",36,4.666667
1125,2301,서울불꽃축제,"['공원', '재미', '엔터테인먼트', '공연', '아이', '싸다, 저렴하다', '관광지']",22,4.363636
1126,2302,하이서울페스티벌,"['예술', '공연', '음식, 식사', '체험', '커플', '문화', '아이']",3,4.666667


In [36]:
sim_order_table = pd.read_csv('유사도 순으로 장소 인덱스를 나열.csv', index_col= '0')
sim_order_table

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
교보문고 광화문,60,78,83,343,120,734,832,863,884,579,181,1015,468,494,570,599,609,529,996,646,666,674,542,110,913,330,838,39,111,66,558,33,445,432,213,...,534,531,530,507,485,487,488,489,490,493,1038,495,1037,1036,503,504,505,506,508,1033,509,510,513,514,516,1035,519,520,521,522,523,524,525,528,586
롯데월드타워&몰,10,1161,147,160,30,217,23,442,472,779,149,419,44,184,853,458,691,715,55,76,79,40,16,56,9,1034,35,7,141,2,641,1020,46,796,50,...,833,831,830,390,388,352,872,353,354,355,356,358,359,360,886,362,363,364,365,366,367,370,885,373,375,376,377,378,379,380,381,382,383,385,586
SM타운 코엑스아티움,10,160,217,779,472,44,147,1010,1161,209,594,131,853,23,715,79,647,56,55,442,46,30,40,1,7,91,387,419,149,981,1013,906,169,675,58,...,823,523,522,521,520,519,824,293,825,826,817,815,541,808,799,569,567,801,802,562,561,806,807,551,814,811,812,951,548,949,546,813,544,543,586
홍대앞 예술시장 프리마켓,24,797,141,685,675,674,15,49,67,47,16,51,217,77,9,6,680,712,30,796,906,149,1013,981,186,692,4,452,1161,187,1009,131,804,853,1168,...,469,459,451,1004,379,380,381,382,383,385,1008,388,389,390,391,394,397,1006,1003,965,409,1001,998,415,997,418,420,994,987,439,971,969,967,966,586
광장시장,47,853,797,51,15,829,692,14,131,16,408,19,25,46,641,140,34,8,647,1013,149,981,906,545,186,1161,179,970,798,904,804,35,90,402,113,...,397,827,826,369,836,367,366,334,335,336,338,339,340,842,342,840,344,347,839,349,350,351,352,353,355,356,837,358,359,361,362,363,364,365,586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
디보 빌리지,1169,215,188,13,610,361,893,899,221,207,727,182,620,612,139,532,168,512,146,164,556,511,124,785,907,99,579,192,574,342,95,83,343,895,496,...,762,765,799,783,798,797,796,795,794,793,792,791,790,789,788,787,786,784,782,766,781,780,779,778,777,776,775,774,773,772,771,770,769,767,586
서울빛초롱축제,137,178,670,205,1032,582,783,400,996,599,666,646,570,1015,529,494,609,468,176,181,286,945,134,653,123,1096,431,297,374,1121,87,952,422,953,943,...,528,530,850,495,904,861,902,894,439,893,888,879,874,455,872,868,461,463,862,471,852,860,859,858,857,481,483,485,486,487,488,490,854,493,586
서울불꽃축제,1116,581,1167,524,874,508,271,1069,1114,423,264,758,1061,1064,680,1115,410,1120,885,425,241,1127,611,970,1009,1007,371,1112,516,1146,594,461,1155,904,198,...,621,354,355,356,620,358,630,332,331,316,304,646,644,307,308,309,310,311,312,314,315,317,330,642,319,640,637,322,324,635,633,327,631,329,586


## tag 리스트 확인해서 사람의 interest 비율 알아보기

In [37]:
tags = pd.read_excel('tag.xlsx')
tag_list = list(tags['FLEX']) + list(tags['교양인']) + list(tags['놀래']) + list(tags['쉴래'])+ list(
    tags['구경할래']) + list(tags['배울래'])
tag_list = [x for x in tag_list if str(x) != 'nan']
tag_list
tag_list

['돈',
 '쇼핑',
 '시장',
 '싸다, 저렴하다',
 '비싸다',
 '먹다',
 '음식, 식사',
 '상품',
 '구매',
 '가격',
 '호텔',
 '박물관',
 '전시관',
 '관광지',
 '유적지',
 '미술관',
 '역사',
 '유익하다',
 '문화',
 '궁전',
 '전쟁',
 '활동',
 '놀이공원',
 '게임',
 '친구',
 '커플',
 '재미',
 '아쿠아리움',
 '엔터테인먼트',
 '아이',
 '가족',
 '경기',
 '힐링',
 '공원',
 '휴식',
 '풍경',
 '자연',
 '전망대',
 '산책',
 '분수',
 '여유',
 '나들이',
 '공연',
 '콘서트',
 '이벤트',
 '영화',
 '예술',
 '미술',
 '강좌',
 '요리',
 '공예',
 '체험']

In [38]:
# tour_type (FLEX / 교양인 / 놀래 / 쉴래 / 구경할래 / 배울래)

# FLEX     ->  type 1
# 교양인   ->  type 2
# 놀래     ->  type 3
# 쉴래     ->  type 4
# 구경할래 ->  type 5
# 배울래   ->  type 6
tags

,FLEX,교양인,놀래,쉴래,구경할래,배울래
0,돈,박물관,활동,힐링,공연,강좌
1,쇼핑,전시관,놀이공원,공원,콘서트,요리
2,시장,관광지,게임,휴식,이벤트,공예
3,"싸다, 저렴하다",유적지,친구,풍경,영화,체험
4,비싸다,미술관,커플,자연,예술,NaN
5,먹다,역사,재미,전망대,미술,NaN
6,"음식, 식사",유익하다,아쿠아리움,산책,NaN,NaN
7,상품,문화,엔터테인먼트,분수,NaN,NaN
8,구매,궁전,아이,여유,NaN,NaN
9,가격,전쟁,가족,나들이,NaN,NaN


## input 을 고려해 가상의 장소 선정

In [39]:
my_series = pd.Series({"여유":15, "힐링":10 , "산책":5})
interest = pd.DataFrame(my_series, columns= ['정도'])
interest['rate'] = interest['정도']/(sum(interest['정도']))
interest

,정도,rate
여유,15,0.500000
힐링,10,0.333333
산책,5,0.166667


In [40]:
# 개인 별 태그의 우선 순위 정하기
interest = interest.sort_values('정도', ascending=False)
user_tag = interest.index.to_list()
user_tag

['여유', '힐링', '산책']

In [43]:
from ast import literal_eval

첫번째태그 = []
두번째태그 = []
세번째태그 = []
네번째태그 = []
다섯번째태그 = []

for i in range(len(review_tag_count)):
    tag = literal_eval(review_tag_count.iloc[i,2])
    try :
        첫번째태그.append(tag[0])
    except :
        print(tag)
        첫번째태그.append('Nan')
        
    try :
        두번째태그.append(tag[1])
    except :
        print(tag)
        두번째태그.append('Nan')
        
    try :
        세번째태그.append(tag[2])
    except :
        print(tag)
        세번째태그.append('Nan')
        
    try :
        네번째태그.append(tag[3])
    except :
        print(tag)
        네번째태그.append('Nan')
        
    try :
        다섯번째태그.append(tag[3])
    except :
        print(tag)
        다섯번째태그.append('Nan')
        
review_tag_count['first_tag'] = 첫번째태그
review_tag_count['second_tag'] = 두번째태그
review_tag_count['third_tag'] = 세번째태그
review_tag_count['fourth_tag'] = 네번째태그
review_tag_count['fifth_tag'] = 다섯번째태그


review_tag_count

['유적지', '친구']
['유적지', '친구']
['유적지', '친구']
['구매', '공예', '가격']
['구매', '공예', '가격']
['전시관', '음식, 식사', '쇼핑']
['전시관', '음식, 식사', '쇼핑']
['구매', '친구', '관광지']
['구매', '친구', '관광지']
['싸다, 저렴하다', '비싸다', '가격']
['싸다, 저렴하다', '비싸다', '가격']
['쇼핑', '음식, 식사', '먹다']
['쇼핑', '음식, 식사', '먹다']
[]
[]
[]
[]
[]
['비싸다', '쇼핑', '영화']
['비싸다', '쇼핑', '영화']
['친구']
['친구']
['친구']
['친구']
[]
[]
[]
[]
[]
['재미', '친구', '게임']
['재미', '친구', '게임']
['시장', '경기']
['시장', '경기']
['시장', '경기']
['시장', '쇼핑', '음식, 식사']
['시장', '쇼핑', '음식, 식사']
['쇼핑', '구매', '가격']
['쇼핑', '구매', '가격']
['궁전', '음식, 식사']
['궁전', '음식, 식사']
['궁전', '음식, 식사']
['재미', '가격']
['재미', '가격']
['재미', '가격']
['박물관', '전시관']
['박물관', '전시관']
['박물관', '전시관']
['전시관']
['전시관']
['전시관']
['전시관']
['쇼핑', '상품']
['쇼핑', '상품']
['쇼핑', '상품']
['시장']
['시장']
['시장']
['시장']
['전시관', '시장', '구매']
['전시관', '시장', '구매']
['쇼핑', '돈', '가격']
['쇼핑', '돈', '가격']
['시장', '음식, 식사', '싸다, 저렴하다']
['시장', '음식, 식사', '싸다, 저렴하다']
['친구']
['친구']
['친구']
['친구']
['커플', '여유']
['커플', '여유']
['커플', '여유']
['관광지']
['관광지']
['관광지']
['관광지']
[]
[]
[]

['공원', '산책']
['공원', '산책']
['공원', '먹다']
['공원', '먹다']
['공원', '먹다']
['먹다']
['먹다']
['먹다']
['먹다']
['역사']
['역사']
['역사']
['역사']
['음식, 식사', '시장']
['음식, 식사', '시장']
['음식, 식사', '시장']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['박물관', '역사']
['박물관', '역사']
['박물관', '역사']
['역사']
['역사']
['역사']
['역사']
['먹다', '공원', '역사']
['먹다', '공원', '역사']
['공예', '예술', '문화']
['공예', '예술', '문화']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['아이']
['아이']
['아이']
['아이']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['먹다', '역사']
['먹다']
['먹다']
['먹다']
['먹다']
[]
[]
[]
[]
[]
['먹다']
['먹다']
['먹다']
['먹다']
['먹다', '음식, 식사']
['먹다', '음식, 식사']
['먹다', '음식, 식사']
['먹다', '공원', '역사']
['먹다', '공원', '역사']
['휴식', '먹다']
['휴식', '먹다']
['휴식', '먹다']
['먹다', '휴식']
['먹다', '휴식']
['먹다', '휴식']
['먹다']
['먹다']
['먹다']
['먹다']
['먹다']
['먹다']
['먹다']
['먹다']
['재미']
['재미']
['재미']
['재미']
['문화']
['문화']
['문화']
['문화']
[]
[]
[]
[]
[]
['공연', '산책']
['공연', '산책']
['공연', '산책']
['궁전', '자연', '역사']
['궁전', '자연', '역사']
['산책', '구매']
['산책

,id,place,tag_list,count,avg_score,first_tag,second_tag,third_tag,fourth_tag,fifth_tag
0,1,교보문고 광화문,"['음식, 식사', '쇼핑', '아이', '구매', '휴식', '비싸다', '문화']",74,4.540541,"음식, 식사",쇼핑,아이,구매,구매
1,2,롯데월드타워&몰,"['상품', '비싸다', '음식, 식사', '싸다, 저렴하다', '쇼핑', '친구', '산책']",203,4.206897,상품,비싸다,"음식, 식사","싸다, 저렴하다","싸다, 저렴하다"
2,3,SM타운 코엑스아티움,"['음식, 식사', '박물관', '상품', '쇼핑', '예술', '싸다, 저렴하다', '비싸다']",66,4.181818,"음식, 식사",박물관,상품,쇼핑,쇼핑
3,4,홍대앞 예술시장 프리마켓,"['음식, 식사', '쇼핑', '시장', '가격', '싸다, 저렴하다', '먹다', '공연']",75,4.386667,"음식, 식사",쇼핑,시장,가격,가격
4,5,광장시장,"['음식, 식사', '시장', '먹다', '싸다, 저렴하다', '가격', '관광지', '돈']",634,4.113565,"음식, 식사",시장,먹다,"싸다, 저렴하다","싸다, 저렴하다"
...,...,...,...,...,...,...,...,...,...,...
1123,2297,디보 빌리지,['쇼핑'],1,1.000000,쇼핑,Nan,Nan,Nan,Nan
1124,2300,서울빛초롱축제,"['음식, 식사', '관광지', '재미', '전시관', '예술', '이벤트', '산책']",36,4.666667,"음식, 식사",관광지,재미,전시관,전시관
1125,2301,서울불꽃축제,"['공원', '재미', '엔터테인먼트', '공연', '아이', '싸다, 저렴하다', '관광지']",22,4.363636,공원,재미,엔터테인먼트,공연,공연
1126,2302,하이서울페스티벌,"['예술', '공연', '음식, 식사', '체험', '커플', '문화', '아이']",3,4.666667,예술,공연,"음식, 식사",체험,체험


In [44]:
review_tag_count.to_csv('data_for_service_1.csv', encoding= 'utf-8-sig', index=False)

## 태그로 가상 장소 찾기

In [21]:
# 키워드가 들어간거 다 뽑기
firstword_table = review_tag_count[review_tag_count['tag_list'].str.contains("여유")]
secondword_table = review_tag_count[review_tag_count['tag_list'].str.contains("힐링")]
thirdword_table = review_tag_count[review_tag_count['tag_list'].str.contains("산책")]

entire_table = pd.concat([firstword_table,secondword_table,thirdword_table])
entire_table = entire_table.drop_duplicates()
entire_table

,id,place,tag_list,listed_tag,tag_list_literal,count,avg_score,first_tag,second_tag,third_tag,fourth_tag,fifth_tag
43,44,메세나폴리스몰,"['쇼핑', '음식, 식사', '비싸다', '먹다', '영화', '싸다, 저렴하다', '여유']","['쇼핑', '음식, 식사', '비싸다', '먹다', '영화', '싸다, 저렴하다', '여유']","쇼핑 음식, 식사 비싸다 먹다 영화 싸다, 저렴하다 여유",55,3.872727,쇼핑,"음식, 식사",비싸다,먹다,먹다
47,48,양재동 화훼공판장,"['시장', '싸다, 저렴하다', '구매', '가격', '먹다', '돈', '여유']","['시장', '싸다, 저렴하다', '구매', '가격', '먹다', '돈', '여유']","시장 싸다, 저렴하다 구매 가격 먹다 돈 여유",21,4.428571,시장,"싸다, 저렴하다",구매,가격,가격
83,84,현대시티아울렛 - 동대문점,"['쇼핑', '음식, 식사', '가격', '먹다', '여유', '관광지', '아이']","['쇼핑', '음식, 식사', '가격', '먹다', '여유', '관광지', '아이']","쇼핑 음식, 식사 가격 먹다 여유 관광지 아이",6,4.000000,쇼핑,"음식, 식사",가격,먹다,먹다
98,100,서울책보고,"['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","여유 문화 싸다, 저렴하다 비싸다 아이 관광지 구매",4,4.250000,여유,문화,"싸다, 저렴하다",비싸다,비싸다
110,113,Gana Art Gallery,"['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']","['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']",예술 전시관 여유 아이 가족 휴식 자연,3,4.666667,예술,전시관,여유,아이,아이
...,...,...,...,...,...,...,...,...,...,...,...,...
1063,2099,거여공원,"['공원', '산책']","['공원', '산책']",공원 산책,1,4.000000,공원,산책,Nan,Nan,Nan
1084,2207,예술의전당 오페라하우스 자유소극장,"['공연', '예술', '산책', '문화', '가족', '아이', '돈']","['공연', '예술', '산책', '문화', '가족', '아이', '돈']",공연 예술 산책 문화 가족 아이 돈,22,4.454545,공연,예술,산책,문화,문화
1086,2209,국립극장,"['공연', '먹다', '음식, 식사', '산책', '예술', '싸다, 저렴하다', '자연']","['공연', '먹다', '음식, 식사', '산책', '예술', '싸다, 저렴하다', '자연']","공연 먹다 음식, 식사 산책 예술 싸다, 저렴하다 자연",19,4.421053,공연,먹다,"음식, 식사",산책,산책
1114,2268,국립중앙도서관,"['음식, 식사', '전시관', '싸다, 저렴하다', '먹다', '돈', '산책', '공원']","['음식, 식사', '전시관', '싸다, 저렴하다', '먹다', '돈', '산책', '공원']","음식, 식사 전시관 싸다, 저렴하다 먹다 돈 산책 공원",9,4.444444,"음식, 식사",전시관,"싸다, 저렴하다",먹다,먹다


In [22]:
A1 = entire_table[entire_table['first_tag'] == "여유"]
A2 = entire_table[entire_table['second_tag'] == "여유"]
A3 = entire_table[entire_table['third_tag'] == "여유"]
important_A = pd.concat([A1,A2,A3])
important_A

,id,place,tag_list,listed_tag,tag_list_literal,count,avg_score,first_tag,second_tag,third_tag,fourth_tag,fifth_tag
98,100,서울책보고,"['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","여유 문화 싸다, 저렴하다 비싸다 아이 관광지 구매",4,4.250000,여유,문화,"싸다, 저렴하다",비싸다,비싸다
146,159,교복몰 - 사당점,"['커플', '여유']","['커플', '여유']",커플 여유,1,5.000000,커플,여유,Nan,Nan,Nan
327,686,김종영미술관,"['전시관', '여유', '미술관', '미술']","['전시관', '여유', '미술관', '미술']",전시관 여유 미술관 미술,1,5.000000,전시관,여유,미술관,미술,미술
506,959,CGV Gonghang Store,"['영화', '여유']","['영화', '여유']",영화 여유,1,3.000000,영화,여유,Nan,Nan,Nan
1038,2070,광나루 한강공원,"['공원', '여유', '먹다']","['공원', '여유', '먹다']",공원 여유 먹다,2,4.500000,공원,여유,먹다,Nan,Nan
110,113,Gana Art Gallery,"['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']","['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']",예술 전시관 여유 아이 가족 휴식 자연,3,4.666667,예술,전시관,여유,아이,아이
414,854,메가박스 목동,"['영화', '쇼핑', '여유', '음식, 식사', '먹다', '미술', '미술관']","['영화', '쇼핑', '여유', '음식, 식사', '먹다', '미술', '미술관']","영화 쇼핑 여유 음식, 식사 먹다 미술 미술관",16,4.312500,영화,쇼핑,여유,"음식, 식사","음식, 식사"


In [27]:
B1 = important_A[important_A['first_tag'] == "힐링"]
B2 = important_A[important_A['second_tag'] == "힐링"]
B3 = important_A[important_A['third_tag'] == "힐링"]
B4 = important_A[important_A['fourth_tag'] == "힐링"]
B5 = important_A[important_A['fifth_tag'] == "힐링"]

important_B = pd.concat([important_A, B1,B2,B3,B4,B5])
important_B = important_B.drop_duplicates()
important_B

,id,place,tag_list,listed_tag,tag_list_literal,count,avg_score,first_tag,second_tag,third_tag,fourth_tag,fifth_tag
98,100,서울책보고,"['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","여유 문화 싸다, 저렴하다 비싸다 아이 관광지 구매",4,4.250000,여유,문화,"싸다, 저렴하다",비싸다,비싸다
146,159,교복몰 - 사당점,"['커플', '여유']","['커플', '여유']",커플 여유,1,5.000000,커플,여유,Nan,Nan,Nan
327,686,김종영미술관,"['전시관', '여유', '미술관', '미술']","['전시관', '여유', '미술관', '미술']",전시관 여유 미술관 미술,1,5.000000,전시관,여유,미술관,미술,미술
506,959,CGV Gonghang Store,"['영화', '여유']","['영화', '여유']",영화 여유,1,3.000000,영화,여유,Nan,Nan,Nan
1038,2070,광나루 한강공원,"['공원', '여유', '먹다']","['공원', '여유', '먹다']",공원 여유 먹다,2,4.500000,공원,여유,먹다,Nan,Nan
110,113,Gana Art Gallery,"['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']","['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']",예술 전시관 여유 아이 가족 휴식 자연,3,4.666667,예술,전시관,여유,아이,아이
414,854,메가박스 목동,"['영화', '쇼핑', '여유', '음식, 식사', '먹다', '미술', '미술관']","['영화', '쇼핑', '여유', '음식, 식사', '먹다', '미술', '미술관']","영화 쇼핑 여유 음식, 식사 먹다 미술 미술관",16,4.312500,영화,쇼핑,여유,"음식, 식사","음식, 식사"


In [28]:
C1 = important_B[important_B['first_tag'] == "산책"]
C2 = important_B[important_B['second_tag'] == "산책"]
C3 = important_B[important_B['third_tag'] == "산책"]
C4 = important_B[important_B['fourth_tag'] == "산책"]
C5 = important_B[important_B['fifth_tag'] == "산책"]

important_C = pd.concat([important_B, C1,C2,C3,C4,C5])
important_C = important_B.drop_duplicates()
important_C

,id,place,tag_list,listed_tag,tag_list_literal,count,avg_score,first_tag,second_tag,third_tag,fourth_tag,fifth_tag
98,100,서울책보고,"['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","여유 문화 싸다, 저렴하다 비싸다 아이 관광지 구매",4,4.250000,여유,문화,"싸다, 저렴하다",비싸다,비싸다
146,159,교복몰 - 사당점,"['커플', '여유']","['커플', '여유']",커플 여유,1,5.000000,커플,여유,Nan,Nan,Nan
327,686,김종영미술관,"['전시관', '여유', '미술관', '미술']","['전시관', '여유', '미술관', '미술']",전시관 여유 미술관 미술,1,5.000000,전시관,여유,미술관,미술,미술
506,959,CGV Gonghang Store,"['영화', '여유']","['영화', '여유']",영화 여유,1,3.000000,영화,여유,Nan,Nan,Nan
1038,2070,광나루 한강공원,"['공원', '여유', '먹다']","['공원', '여유', '먹다']",공원 여유 먹다,2,4.500000,공원,여유,먹다,Nan,Nan
110,113,Gana Art Gallery,"['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']","['예술', '전시관', '여유', '아이', '가족', '휴식', '자연']",예술 전시관 여유 아이 가족 휴식 자연,3,4.666667,예술,전시관,여유,아이,아이
414,854,메가박스 목동,"['영화', '쇼핑', '여유', '음식, 식사', '먹다', '미술', '미술관']","['영화', '쇼핑', '여유', '음식, 식사', '먹다', '미술', '미술관']","영화 쇼핑 여유 음식, 식사 먹다 미술 미술관",16,4.312500,영화,쇼핑,여유,"음식, 식사","음식, 식사"


In [161]:
name3 = important_B[important_B['place'].str.contains("책")]

print(len(name3))

name_vr = []
id_vr = []

for i in range(len(name3)):
    id_fk = name3.iloc[i,0]
    name_fk = name3.iloc[i,1]
    id_vr.append(id_fk)
    name_vr.append(name_fk)
name_vr

1


['서울책보고']

In [167]:
sim_order_table = pd.read_csv('유사도 순으로 장소 인덱스를 나열.csv')
recom = sim_order_table[sim_order_table['0']=='서울책보고'].iloc[:,:3]
sim_order_table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173
0,교보문고 광화문,60,78,83,343,120,734,832,863,884,579,181,1015,468,494,570,599,609,529,996,646,666,674,542,110,913,330,838,39,111,66,558,33,445,432,...,534,531,530,507,485,487,488,489,490,493,1038,495,1037,1036,503,504,505,506,508,1033,509,510,513,514,516,1035,519,520,521,522,523,524,525,528,586
1,롯데월드타워&몰,10,1161,147,160,30,217,23,442,472,779,149,419,44,184,853,458,691,715,55,76,79,40,16,56,9,1034,35,7,141,2,641,1020,46,796,...,833,831,830,390,388,352,872,353,354,355,356,358,359,360,886,362,363,364,365,366,367,370,885,373,375,376,377,378,379,380,381,382,383,385,586
2,SM타운 코엑스아티움,10,160,217,779,472,44,147,1010,1161,209,594,131,853,23,715,79,647,56,55,442,46,30,40,1,7,91,387,419,149,981,1013,906,169,675,...,823,523,522,521,520,519,824,293,825,826,817,815,541,808,799,569,567,801,802,562,561,806,807,551,814,811,812,951,548,949,546,813,544,543,586
3,홍대앞 예술시장 프리마켓,24,797,141,685,675,674,15,49,67,47,16,51,217,77,9,6,680,712,30,796,906,149,1013,981,186,692,4,452,1161,187,1009,131,804,853,...,469,459,451,1004,379,380,381,382,383,385,1008,388,389,390,391,394,397,1006,1003,965,409,1001,998,415,997,418,420,994,987,439,971,969,967,966,586
4,광장시장,47,853,797,51,15,829,692,14,131,16,408,19,25,46,641,140,34,8,647,1013,149,981,906,545,186,1161,179,970,798,904,804,35,90,402,...,397,827,826,369,836,367,366,334,335,336,338,339,340,842,342,840,344,347,839,349,350,351,352,353,355,356,837,358,359,361,362,363,364,365,586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,디보 빌리지,1169,215,188,13,610,361,893,899,221,207,727,182,620,612,139,532,168,512,146,164,556,511,124,785,907,99,579,192,574,342,95,83,343,895,...,762,765,799,783,798,797,796,795,794,793,792,791,790,789,788,787,786,784,782,766,781,780,779,778,777,776,775,774,773,772,771,770,769,767,586
1170,서울빛초롱축제,137,178,670,205,1032,582,783,400,996,599,666,646,570,1015,529,494,609,468,176,181,286,945,134,653,123,1096,431,297,374,1121,87,952,422,953,...,528,530,850,495,904,861,902,894,439,893,888,879,874,455,872,868,461,463,862,471,852,860,859,858,857,481,483,485,486,487,488,490,854,493,586
1171,서울불꽃축제,1116,581,1167,524,874,508,271,1069,1114,423,264,758,1061,1064,680,1115,410,1120,885,425,241,1127,611,970,1009,1007,371,1112,516,1146,594,461,1155,904,...,621,354,355,356,620,358,630,332,331,316,304,646,644,307,308,309,310,311,312,314,315,317,330,642,319,640,637,322,324,635,633,327,631,329,586
1172,하이서울페스티벌,1154,908,114,348,500,205,884,599,666,570,609,181,529,494,468,996,1015,646,684,330,1140,1121,1119,749,1133,948,959,240,457,930,955,990,601,1173,...,575,574,573,569,568,567,564,563,562,556,555,554,550,549,580,583,584,597,608,607,604,603,602,600,598,596,585,595,594,593,592,590,589,588,586


In [45]:
sim_order_table = pd.read_csv('유사도 순으로 장소 인덱스를 나열.csv')
sim_order_table = sim_order_table.iloc[:,:10]
sim_order_table.to_csv('유사도 순으로 장소 인덱스를 나열_10개만.csv', encoding = 'utf-8-sig', index=False)

In [174]:
recom = sim_order_table[sim_order_table['0']=='서울책보고'].iloc[:,:2]


,0,1,2
103,서울책보고,581,159


In [138]:
B1 = review_tag_count[review_tag_count['first_tag'] == "힐링"]



C1 = review_tag_count[review_tag_count['first_tag'] == "책"]
A2 = review_tag_count[review_tag_count['second_tag'] == "여유"]
B2 = review_tag_count[review_tag_count['second_tag'] == "힐링"]
C2 = review_tag_count[review_tag_count['second_tag'] == "책"]
A3 = review_tag_count[review_tag_count['third_tag'] == "여유"]
B3 = review_tag_count[review_tag_count['third_tag'] == "힐링"]
C3 = review_tag_count[review_tag_count['third_tag'] == "책"]

important_table = pd.concat([A1,B1,C1,A2,B2,C2,A3,B3,C3])
important_table = important_table.drop_duplicates()
important_table

,id,place,tag_list,listed_tag,tag_list_literal,count,avg_score,first_tag,second_tag,third_tag,fourth_tag
98,100,서울책보고,"['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","['여유', '문화', '싸다, 저렴하다', '비싸다', '아이', '관광지', '구매']","여유 문화 싸다, 저렴하다 비싸다 아이 관광지 구매",4,4.250000,여유,문화,"싸다, 저렴하다",비싸다
577,1195,약손119마사지,"['힐링', '돈', '먹다']","['힐링', '돈', '먹다']",힐링 돈 먹다,3,3.666667,힐링,돈,먹다,Nan
610,1237,마르지아힐링스파 청담점,"['힐링', '체험', '가격', '커플', '친구', '호텔', '자연']","['힐링', '체험', '가격', '커플', '친구', '호텔', '자연']",힐링 체험 가격 커플 친구 호텔 자연,13,4.923077,힐링,체험,가격,커플
665,1471,길상사,"['힐링', '박물관', '휴식', '음식, 식사', '역사', '자연', '산책']","['힐링', '박물관', '휴식', '음식, 식사', '역사', '자연', '산책']","힐링 박물관 휴식 음식, 식사 역사 자연 산책",25,4.720000,힐링,박물관,휴식,"음식, 식사"
825,1651,호압사,['힐링'],['힐링'],힐링,1,3.000000,힐링,Nan,Nan,Nan
146,159,교복몰 - 사당점,"['커플', '여유']","['커플', '여유']",커플 여유,1,5.000000,커플,여유,Nan,Nan
327,686,김종영미술관,"['전시관', '여유', '미술관', '미술']","['전시관', '여유', '미술관', '미술']",전시관 여유 미술관 미술,1,5.000000,전시관,여유,미술관,미술
506,959,CGV Gonghang Store,"['영화', '여유']","['영화', '여유']",영화 여유,1,3.000000,영화,여유,Nan,Nan
1038,2070,광나루 한강공원,"['공원', '여유', '먹다']","['공원', '여유', '먹다']",공원 여유 먹다,2,4.500000,공원,여유,먹다,Nan
534,1132,프로방스 스파 바이 록시땅,"['돈', '힐링', '휴식', '싸다, 저렴하다']","['돈', '힐링', '휴식', '싸다, 저렴하다']","돈 힐링 휴식 싸다, 저렴하다",12,4.750000,돈,힐링,휴식,"싸다, 저렴하다"
